<center><h1 style="color:#0098E9;"> Topicos avanzados de Ingenieria Matematica</h1></center>
<center><h1 style="color:#0098E9;"> Tarea 2</h1></center>
<center><h3 style="color:#0098E9;"> Manuel A. Sanchez</h3></center>
<center><h3 style="color:#0098E9;"> Pontificia Universidad Catolica de Chile</h3></center>
<hr style="border:2px solid gray"> </hr>

## 3. Metodo de HDG en una dimension

Considere el problema de Poisson en el intervalo $(-1,1)$
\begin{equation*}
\begin{array}{rclrl}
\displaystyle
-\frac{d}{dx}(c^{-1}(x) \frac{d}{dx} u(x)) & = & f(x)& \mbox{en} & (-1,1) \\
\displaystyle
u & = & u_{D} & \mbox{sobre}& \{-1,1\}.
\end{array}
\end{equation*}

- 3.1. Programe el m\'etodo de HDG para el problema de Poisson utilizando espacios locales $V(I_i)  =  \mathcal P_{p}$,
$W(I_i) =  \mathcal P_{p}$, and $ M(\partial I_i) =L^{2}(\{x_i\}_{i=0}^{N})$ para $p=0,1,2,3$ y par\'ametro de estabilizaci\'on $\tau>0$. 
- 3.2 Testee su programa para la soluci\'on exacta $u(x) = \sin(x)$ and $c(x) = x^{2}+1$.
- 3.3 Reporte, ya sea en una tabla o un gr\'afico log-log, los errores para las aproximaciones de $u$, $q$ y $u|_{\mathcal F_h}$ y los respectivos ordenes de convergencia.  



## Respuesta.

In [1043]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import scipy.sparse as sps
from scipy.sparse.linalg import spsolve
from scipy.integrate import quadrature
from numpy import sin, cos, log
from scipy.special import comb
import time
from scipy.special import roots_jacobi

In [1044]:
# Algorithm Compute_binomial_coefficients
# Input : N
# Output: all binomial coefficients binom(n, j) for j=0,..,n, for n=0,...,N 
def Compute_binomial_coefficients(N):
    # compute all binomial coefficients
    Cb = []
    for n in range(0,N+1):
        c = []
        for j in range(n+1):
            c.append(comb(n,j))
        # end for
        Cb.append(c)
    return Cb

# Algorithm Convective1d
# Input : K, n
# Output: local convective matrix using polynmials of degree n
#         int_K B_j^{n}(x) * d/dx B_i^{n}(x) dx
def Convective1d(K, n):
    # Compute convective matrix 
    if n == 0:
        return np.zeros((n+1,n+1), dtype=np.float64) 
    d = 1
    # grad \lambda_k = - |\gamma_k|/(d|K|) n_{K}
    hK = abs(K[1]-K[0])
    gradlambda = [-1.0/hK, 1.0/hK]
    mutilde = np.zeros((2*n-2+2,d+1))
    muA = Compute_mu(K, 2*n-1)
    for l in range(0, d+1):
        mutilde[:,l] = muA*gradlambda[l]
    B = np.zeros((n+1,n+1), dtype=np.float64)
    for alpha0 in range(0,n+1):
        for beta0 in range(0,n):
            w0 = Cb[alpha0+beta0][alpha0]/Cb[2*n-1][n-1] #comb(alpha0+beta0, alpha0)/comb(2*n-1, n-1)
            w1 = w0*Cb[(n)-alpha0 + (n-1)-beta0][(n)-alpha0] #w0*comb((n-1)-alpha0+(n)-beta0, (n-1)-alpha0)
            for l in range(0,d+1):
                B[alpha0,beta0+l] += n*w1*mutilde[alpha0+beta0,l]
    return B

# Algorithm Mass1d
# Input : K, n, c
# Output: local mass matrix using polynmials of degree n
#         int_K B_j^{n}(x) * c(x) * B_i^{n}(x) dx
def Mass1d(K, n, c):
    d = 1
    mu = Compute_mu(K, 2*n,c)
    M = np.zeros((n+1,n+1), dtype=np.float64)
    for alpha0 in range(0,n+1):
        for beta0 in range(0,n+1):
            w0 = Cb[alpha0+beta0][alpha0]/Cb[2*n][n] #comb(alpha0+beta0, alpha0)/comb(2*n, n)
            w1 = w0*Cb[(n)-alpha0 + (n)-beta0][(n)-alpha0]#w0*comb((n)-alpha0+(n)-beta0, (n)-alpha0)
            M[alpha0,beta0] += w1*mu[alpha0+beta0]
    return M

# Algortihm  Compute_mu
# Input : element K, n, f
# Output: Bernstein Bezier moments of degree n of a funcion f
from math import ceil
def Compute_mu(K, n, f=None, q=None):
    # compute the Bernstein Bezier moments of a function f
    # dimension
    d = 1
    # number of quadrature points
    if q is None or q <= 0.5*(n+1):
        q = ceil(0.5*(n+1))
    
    if f is None:
        F0 = np.ones(q)
        mu = abs(K[1]-K[0])/d*Moment(F0,q,n)
    else:
        # Gauss-Jacobi points
        xiv, omegav = roots_jacobi(q, alpha=0, beta=0, mu=False)
        # transformation from (-1,1) to K = (a,b)
        xK = 0.5*(K[0]*(1-xiv) + K[1]*(1+xiv))
        mu = abs(K[1]-K[0])/d*Moment(f(xK), q,n)
    return mu

# Algorithm Moment(F0, q)
# Input : Array F0 corresponding to values of a function f at Stroud nodes
# Output: Array Fd composed of Bernstein-Bezier moments of f using Stroud rule
def Moment(F0, q,n):
    d = 1
    Fnew = F0.copy()
    for l in range(1, d+1):
        Fold = Fnew
        Fnew = MomentStep(Fold, l, q,n)
    return Fnew


# Algorithm MomentStep(Fin, l, q)
# Input : Array Fin and index l in {1,...,d}
# Output: Updated array Fout with index il switched to alphal
def MomentStep(Fin, l, q, n):
    xiv, omegav = roots_jacobi(q, alpha=0, beta=0, mu=False)
    xiv += 1.0
    xiv *= 0.5
    omegav *= 0.5
    
    Fout = np.zeros(n+1)
    for il in range(0,q):
        xi    = xiv[il]
        omega = omegav[il]
        s     = 1-xi
        r     = xi/s
        w = omega*s**n
        for alphal in range(0, n+1):
            Fout[alphal] += w*Fin[il]
            w *= r*(n-alphal)/(1+alphal)
    return Fout

# Algorithm for interpolation
def NewtonBernstein(f, x=None):
    n = f.size-1
    if x is None:
        x = np.linspace(0, 1, n+1, endpoint=True)
    # end if

    c = np.zeros(n+1, dtype = np.float64)
    w = np.zeros(n+1, dtype = np.float64)

    c[0] = f[0]
    w[0] = 1.0
    for s in range(1,n+1):
        for k in range(n, s-1, -1):
            f[k] = (f[k]-f[k-1])/(x[k]-x[k-s])
        # end for
        for k in range(s, 0,-1):
            w[k] = (k/s)*w[k-1]*(1-x[s-1]) - (1-k/s)*w[k]*x[s-1]
            c[k] = (k/s)*c[k-1] + (1-k/s)*c[k] + f[s]*w[k]
        # end for
        w[0] = -w[0]*x[s-1]
        c[0] = c[0] + f[s]*w[0]
    # end for
    return c

In [1045]:
# Algorithm hdg1d_Poisson
# Description: Simple algortihm for computing the finite element continuous Galerkin matrix
# Input : Coordinate, Elements, NN, NE, f, c, p
# Output: global stiffness matrix A and load vector
def hdg1d_Poisson_hybridization(mesh, p=1, f=None, c=None):
    iA = []; jA = []; kA = [];
    iR = []; jR = []; kR = [];
    # precompute the binomial coefficients
    global Cb 
    Cb = Compute_binomial_coefficients(max(2*p,p))
    # Load vector
    b = np.zeros(mesh.NN);
    Rb = np.zeros(2*mesh.NE*(p+1))
    # General Structure of degrees of freedom: [Nodes, faces, interior]
    for j in range(mesh.NE):
        # Interval
        K = mesh.Coordinates[mesh.Elements[j,:]]
    
        # Local Matrices
        Bi = Convective1d(K, p)
        Mi = Mass1d(K, p, c)
        Si = np.zeros((p+1,p+1)); Si[0,0] = 1.0; Si[p,p] = 1.0
        Ei = np.zeros((2,p+1))  ; Ei[0,0] = 1.0; Ei[1,p] = 1.0
        Ci = np.zeros((2,p+1))  ; Ci[0,0] = -1.0; Ci[1,p] = 1.0
        Gi = np.eye(2)
        
        # Local load vector
        if f is None:
            Fi = np.zeros(p+1)
        else:
            Fi = Compute_mu(K, p, f)
        
        # condensation
        Qp = np.vstack(( np.hstack(( Mi, -Bi.T)),  np.hstack(( Bi, Si)) ))
        CEi = np.hstack(( Ci, Ei ))
        Ri = np.linalg.solve(Qp, np.vstack(( Ci.T, -Ei.T )))
        Ai =  -CEi @ Ri - Gi
        Rbi = np.linalg.solve(Qp, np.hstack((np.zeros(p+1), Fi )))
        bi = -CEi @ Rbi
        
        # Global Assembling
        dofn = mesh.Elements[j,:]
        # assembling b
        b[dofn] += bi
        # Sparse assembling of A: A[dof,dof] = AK
        rows = dofn; cols = dofn
        prod = [(x,y) for x in rows for y in cols];
        iAlocal = [x for (x,y) in prod]
        jAlocal = [y for (x,y) in prod]
        kAlocal = np.reshape(Ai, (1,Ai.size))
        iA.extend(iAlocal); jA.extend(jAlocal); kA.extend(kAlocal.flatten())
        
        # Reconstruction
        # interior degrees of freedom
        dofq = np.arange((p+1)*(j), (p+1)*(j+1)).tolist()
        dofu = np.arange(mesh.NE*(p+1)+(p+1)*(j), mesh.NE*(p+1)+(p+1)*(j+1)).tolist()
        dofi = dofq+dofu
        Rb[dofi] = Rbi
        # Sparse assembling of A: A[dof,dof] = AK
        rows = dofi; cols = dofn
        prod = [(x,y) for x in rows for y in cols];
        iRlocal = [x for (x,y) in prod]
        jRlocal = [y for (x,y) in prod]
        kRlocal = np.reshape(Ri, (1,Ri.size))
        iR.extend(iRlocal); jR.extend(jRlocal); kR.extend(kRlocal.flatten())
    # end for
    A = sps.coo_matrix(( kA,(iA, jA)), (mesh.NN, mesh.NN), dtype=np.float64)
    R = sps.coo_matrix(( kR,(iR, jR)), (2*mesh.NE*(p+1), mesh.NN), dtype=np.float64)
    #fig, axa = plt.subplots(1,1,figsize=(4,4))
    #axa.spy(A, markersize=5)
    #plt.show()
    return (A, b, R, Rb)

In [1046]:
def hdg1d_solver(mesh, p=1, f=None, c=None, uD=None, hybridization=True):
    # Solve linear system
    if hybridization:
        (A, b, R, Rb) = hdg1d_Poisson_hybridization(mesh, p, f, c)
        uhat = np.zeros(mesh.NN);
        uhat[0] = uD(mesh.Coordinates[0])
        uhat[mesh.NN-1] = uD(mesh.Coordinates[-1])
        b = b- A.dot(uhat)
        All = range(0,mesh.NN)
        Free = np.setdiff1d(All,[0,mesh.NN-1]);
        AFree = A.tocsc()[Free, :][:, Free]
        bFree = b[Free]
        uhat[Free] = spsolve(AFree, bFree)
        
        qu = Rb-R.dot(uhat)
        qh = qu[0:mesh.NE*(p+1)]
        uh = qu[mesh.NE*(p+1):mesh.NE*2*(p+1)]
        return qh, uh, uhat
    else:
        (A, b) = fem1d_Poisson(mesh, p, f, c)
        uh = np.zeros(mesh.NN+(p-1)*mesh.NE);
        All = range(0,mesh.NN+(p-1)*mesh.NE)
        Free = np.setdiff1d(All,[0,mesh.NN-1]);
        AFree = A.tocsc()[Free, :][:, Free]
        print("*** full system: size of A", AFree.shape)
        bFree = b[Free]
        uh[Free] = spsolve(AFree, bFree)
        return uh

In [1047]:
# Triangulation
class mesh1d:
    def __init__(self, Coordinates, Elements):
        self.Coordinates = Coordinates
        self.Elements = Elements
        self.NE = Elements.shape[0] # number of elements
        self.NN = Coordinates.size # number of coordinates or nodes
        self.h = np.max([abs(Coordinates[j+1] - Coordinates[j]) for j in range(self.NE)]) # parameter of the triangulation

def equispaced_mesh(xi,xf,j):
    n = 2**j;
    Coordinates = np.linspace(xi,xf,num = n+2) # nodes 
    Elements = np.vstack([range(n+1), range(1,n+2)]).T
    return mesh1d(Coordinates, Elements)

In [1069]:
import matplotlib.pyplot as plt
import numpy as np
def plot_hdg1d(mesh, p, qh, uh, uhat, u, q):
    plt.rc('text', usetex=True)
    fig, ax = plt.subplots(2,1,figsize=(10, 14))
    # uhat and uh plot
    line_uhat,=ax[0].plot(mesh.Coordinates, uhat, color='C1',marker='o', markersize = 6, linestyle='None', label=r'$\widehat{u}$')
    for j in range(mesh.NE):
        dof = [j*(p+1), (j+1)*(p+1) -1]
        xx = mesh.Coordinates[mesh.Elements[j,:]]
        line_uh, = ax[0].plot(xx, uh[dof], color='C2')
    line_uh.set_label(r'$u_h$')
    xplot = np.linspace(mesh.Coordinates[0],mesh.Coordinates[-1],200)
    line_uexac = ax[0].plot(xplot, u(xplot), 'C3--', label = r'$u$ exact')
    ax[0].legend(loc='upper right', prop={'size': 18})

    # qh plot
    for j in range(mesh.NE):
        dof = [j*(p+1), (j+1)*(p+1) -1]
        xx = mesh.Coordinates[mesh.Elements[j]]
        line_qh, = ax[1].plot(xx, qh[dof], color='C0')
    line_qh.set_label(r'$q_h$')
    line_qexac = ax[1].plot(xplot, q(xplot), 'C3--', label = r'$q$ exact')
    ax[1].legend(loc='upper right',prop={'size': 18})
    
    plt.show()    

In [1084]:
def errors_hdg1d(mesh, p, c, uh, qh, uhat, u, q):
    el2u = 0
    el2q = 0
    for j in range(mesh.NE):
        K = mesh.Coordinates[mesh.Elements[j,:]]
        
        xx = np.linspace(K[0],K[1], p+1 )
        uj = NewtonBernstein(u(xx))
        uhj = uh[j*(p+1):(j+1)*(p+1)]
        Mj = Mass1d(K, p, None)
        el2u += (uj-uhj).dot(Mj.dot(uj-uhj))
        
        Mjc = Mass1d(K, p, c)
        qj = NewtonBernstein(q(xx))
        qhj = qh[j*(p+1):(j+1)*(p+1)]
        el2q += (qj-qhj).dot(Mj.dot(qj-qhj))
    euhat = (u(mesh.Coordinates)-uhat).dot(u(mesh.Coordinates)-uhat)
    return np.sqrt(el2u), np.sqrt(el2q), np.sqrt(mesh.h*euhat)

from pandas import DataFrame
def print_markdown_table_amr(errors):
    i = 1
    h = []
    error_u = []
    order_u = []
    error_q = []
    order_q = []
    error_uhat = []
    order_uhat = []
    while i < len(errors):
        h.append(errors[i][0])
        error_u.append(errors[i][1])
        order_u.append(log(errors[i-1][1]/errors[i][1])/log(errors[i-1][0]/errors[i][0]))
        error_q.append(errors[i][2])
        order_q.append(log(errors[i-1][2]/errors[i][2])/log(errors[i-1][0]/errors[i][0]))
        error_uhat.append(errors[i][3])
        order_uhat.append( log(errors[i-1][3]/errors[i][3])/log(errors[i-1][0]/errors[i][0]))
        i += 1
    # pretty print
    df = DataFrame({"h":h,'$$\|u-u_h\|_{L^2}$$': error_u,"$$\mbox{order } u$$": order_u, "$$\|q-q_h\|_{L^2}$$": error_q, " $$\mbox{order }q$$": order_q, "$$|u-\widehat{u}_h|_{\ell^2}$$": error_uhat, " $$\mbox{order }\widehat{u}_h$$": order_uhat})
    display(df)

In [1085]:
def convergence_test(p = 1, last_mesh=2, plot=False):
    xi = -1; xf = 1;
    u = lambda x: sin(x)
    du = lambda x: cos(x)
    q = lambda x: -cos(x)/(x**2+1)
    c  = lambda x: x**2+1
    f = lambda x: (sin(x)*(x**2+1)+2*x*cos(x))/(x**2+1)**2
    
    output = []
    for i in range(0,last_mesh):
        mesh = equispaced_mesh(xi,xf,i)
        qh, uh, uhat = hdg1d_solver(mesh, p=p, f=f, c=c, uD=u, hybridization=True)
        eu, eq, euhat = errors_hdg1d(mesh, p, c, uh, qh, uhat, u, q)
        output.append (( mesh.h,  eu, eq, euhat))
    if plot:
        plot_hdg1d(mesh, p, qh, uh, uhat, u, q)
    return output

## Test $p=0$:

In [1079]:
output = convergence_test(p=0, last_mesh=8, plot=False)
print_markdown_table_amr(output)

,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$|u-\widehat{u}_h|_{\ell^2}$$,$$\mbox{order }\widehat{u}_h$$
0,0.666667,2.131015e-03,5.077401,3.828726e-03,1.906971,2.202383e-03,-73.806120
1,0.400000,4.078919e-04,3.236625,4.179269e-04,4.336080,1.587671e-04,5.148248
2,0.222222,6.457046e-05,3.135909,5.345520e-05,3.498655,1.352182e-05,4.190523
3,0.117647,9.415216e-06,3.027459,7.377426e-06,3.113921,1.036408e-06,4.038663
4,0.060606,1.285753e-06,3.001658,9.948532e-07,3.020658,7.247470e-08,4.010706
5,0.030769,1.685169e-07,2.997676,1.302204e-07,2.999598,4.805306e-09,4.002948
6,0.015504,2.158745e-08,2.998028,1.669363e-08,2.996974,3.095781e-10,4.000825


## Test $p=1$:

In [1080]:
output = convergence_test(p=1, last_mesh=8, plot=False)
print_markdown_table_amr(output)

,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$|u-\widehat{u}_h|_{\ell^2}$$,$$\mbox{order }\widehat{u}_h$$
0,0.666667,0.016120,3.046596,0.029442,2.765831,0.025623,-82.588004
1,0.400000,0.005577,2.077887,0.015159,1.299439,0.005999,2.842290
2,0.222222,0.001830,1.895865,0.005074,1.862067,0.001676,2.169341
3,0.117647,0.000535,1.933318,0.001472,1.945753,0.000454,2.053920
4,0.060606,0.000145,1.964402,0.000397,1.976482,0.000119,2.017773
5,0.030769,0.000038,1.981786,0.000103,1.989219,0.000031,2.006218
6,0.015504,0.000010,1.990808,0.000026,1.994861,0.000008,2.002387


## Test $p=2$:

In [1081]:
output = convergence_test(p=2, last_mesh=8, plot=False)
print_markdown_table_amr(output)

,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$|u-\widehat{u}_h|_{\ell^2}$$,$$\mbox{order }\widehat{u}_h$$
0,0.666667,2.131015e-03,5.077401,3.828726e-03,1.906971,2.202383e-03,-73.806120
1,0.400000,4.078919e-04,3.236625,4.179269e-04,4.336080,1.587671e-04,5.148248
2,0.222222,6.457046e-05,3.135909,5.345520e-05,3.498655,1.352182e-05,4.190523
3,0.117647,9.415216e-06,3.027459,7.377426e-06,3.113921,1.036408e-06,4.038663
4,0.060606,1.285753e-06,3.001658,9.948532e-07,3.020658,7.247470e-08,4.010706
5,0.030769,1.685169e-07,2.997676,1.302204e-07,2.999598,4.805306e-09,4.002948
6,0.015504,2.158745e-08,2.998028,1.669363e-08,2.996974,3.095781e-10,4.000825


## Test $p=3$:

In [1082]:
output = convergence_test(p=3, last_mesh=8, plot=False)
print_markdown_table_amr(output)

,h,$$\|u-u_h\|_{L^2}$$,$$\mbox{order } u$$,$$\|q-q_h\|_{L^2}$$,$$\mbox{order }q$$,$$|u-\widehat{u}_h|_{\ell^2}$$,$$\mbox{order }\widehat{u}_h$$
0,0.666667,0.023355,-0.110106,0.028383,1.212016,1.883318e-03,-74.419224
1,0.400000,0.007496,2.224628,0.008940,2.261652,1.260989e-04,5.292851
2,0.222222,0.002288,2.019082,0.002687,2.045079,1.046879e-05,4.233966
3,0.117647,0.000636,2.011892,0.000747,2.012811,7.990189e-07,4.045305
4,0.060606,0.000168,2.007658,0.000198,2.003577,5.581973e-08,4.012182
5,0.030769,0.000043,2.004338,0.000051,2.000945,3.700443e-09,4.003181
6,0.015504,0.000011,2.002307,0.000013,2.000243,2.383979e-10,4.000826
